<a href="https://colab.research.google.com/github/Taehwan2/hanghaeAI/blob/main/DistilBERT%EC%8B%AC%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/unlocking-language-understanding-with-the-multin")

print("Path to dataset files:", path)

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# DistilBERT 모델용 tokenizer 로드 (pretrained)
# 이 tokenizer는 문장을 토큰화해서 모델이 이해할 수 있는 input_ids로 변환해줌
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [11]:
import random
import pandas as pd


def load_data(path, nrows=None):
  df = pd.read_csv(path, nrows=nrows, keep_default_na=False)
  data = []
  for _, row in df.iterrows():
    if len(row['premise']) * len(row['hypothesis']) != 0:
      data.append({'premise': row['premise'], 'hypothesis': row['hypothesis'], 'label': row['label']})

  return data


train_data = load_data(path + '/train.csv', nrows=1000)
test_data = load_data(path + '/validation_matched.csv', nrows=1000)

In [12]:
def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['premise'] + row['hypothesis'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    train_data, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_data, batch_size=64, shuffle=False, collate_fn=collate_fn
)

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [ ]:
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, 3)

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x


model = TextClassifier()

위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [14]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [15]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').long()

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch   0 | Train Loss: 17.51239264011383
Epoch   1 | Train Loss: 17.373963832855225
Epoch   2 | Train Loss: 17.33500647544861
Epoch   3 | Train Loss: 17.22771990299225
Epoch   4 | Train Loss: 17.132736086845398
Epoch   5 | Train Loss: 16.957014560699463
Epoch   6 | Train Loss: 16.862652897834778
Epoch   7 | Train Loss: 16.926201343536377
Epoch   8 | Train Loss: 16.894115805625916
Epoch   9 | Train Loss: 16.796897888183594
Epoch  10 | Train Loss: 16.855460166931152
Epoch  11 | Train Loss: 16.750187277793884
Epoch  12 | Train Loss: 16.622865319252014
Epoch  13 | Train Loss: 16.728083729743958
Epoch  14 | Train Loss: 16.539232850074768
Epoch  15 | Train Loss: 16.556726038455963
Epoch  16 | Train Loss: 16.40929800271988
Epoch  17 | Train Loss: 16.47456306219101
Epoch  18 | Train Loss: 16.42001336812973
Epoch  19 | Train Loss: 16.29828578233719
Epoch  20 | Train Loss: 16.219700694084167
Epoch  21 | Train Loss: 16.21201992034912
Epoch  22 | Train Loss: 16.418890058994293
Epoch  23 | Train L

In [16]:
def accuracy(model, dataloader):
    cnt = 0      # 전체 샘플 수
    acc = 0      # 정답 개수 누적

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        preds = model(inputs)  # 로짓(logit) 출력

        # 시그모이드는 생략 가능 (BCEWithLogitsLoss를 썼다면 threshold만 적용)
        preds = torch.argmax(preds, dim=-1)
        #preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]  # 총 샘플 수 누적
        acc += (labels == preds).sum().item()  # 예측이 맞은 수 누적

    return acc / cnt  # 정확도 반환

# 평가 시 gradient 계산 비활성화
with torch.no_grad():
    model.eval()  # 평가 모드로 전환 (계산 비활성화)
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)

    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")


=========> Train acc: 0.549 | Test acc: 0.397


In [ ]:
### 사전학습 된거 가중치 제거

In [17]:
import torch
from torch import nn

class TextClassifier2(nn.Module):
    def __init__(self):
        super().__init__()

        # 사전학습된 DistilBERT 모델을 encoder로 불러옴
        self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
        self.classifier = nn.Linear(768, 3)

        # 모든 가중치 초기화
        self._initialize_weights()

    def _initialize_weights(self):
        # DistilBERT의 모든 파라미터를 랜덤 초기화
        for module in self.encoder.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # Xavier 초기화
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.uniform_(module.weight, -0.1, 0.1)  # 임베딩 초기화

        # 분류기의 가중치도 초기화
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, x):
        x = self.encoder(x)['last_hidden_state']
        x = self.classifier(x[:, 0])
        return x

# 모델 생성
model2 = TextClassifier2()


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [18]:
for param in model2.encoder.parameters():
  param.requires_grad = False

In [19]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model2 = model2.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model2.parameters(), lr=lr)
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model2.train()
  for data in train_loader:
    model2.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').long()

    preds = model2(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 18.11597728729248
Epoch   1 | Train Loss: 17.97783398628235
Epoch   2 | Train Loss: 17.89479374885559
Epoch   3 | Train Loss: 17.576369166374207
Epoch   4 | Train Loss: 17.628673791885376
Epoch   5 | Train Loss: 18.12171733379364
Epoch   6 | Train Loss: 17.864755988121033
Epoch   7 | Train Loss: 17.660717368125916
Epoch   8 | Train Loss: 17.61660075187683
Epoch   9 | Train Loss: 17.635198831558228
Epoch  10 | Train Loss: 17.56428360939026
Epoch  11 | Train Loss: 17.80129587650299
Epoch  12 | Train Loss: 17.606622219085693
Epoch  13 | Train Loss: 17.527899861335754
Epoch  14 | Train Loss: 17.646907567977905
Epoch  15 | Train Loss: 17.522242784500122
Epoch  16 | Train Loss: 17.705416440963745
Epoch  17 | Train Loss: 17.438100814819336
Epoch  18 | Train Loss: 17.551884174346924
Epoch  19 | Train Loss: 17.564248919487
Epoch  20 | Train Loss: 17.431082725524902
Epoch  21 | Train Loss: 17.392606258392334
Epoch  22 | Train Loss: 17.447736859321594
Epoch  23 | Train Los

In [21]:
def accuracy(model, dataloader):
    cnt = 0      # 전체 샘플 수
    acc = 0      # 정답 개수 누적

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        preds = model(inputs)  # 로짓(logit) 출력

        # 시그모이드는 생략 가능 (BCEWithLogitsLoss를 썼다면 threshold만 적용)
        preds = torch.argmax(preds, dim=-1)
        #preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]  # 총 샘플 수 누적
        acc += (labels == preds).sum().item()  # 예측이 맞은 수 누적

    return acc / cnt  # 정확도 반환

# 평가 시 gradient 계산 비활성화
with torch.no_grad():
    model.eval()  # 평가 모드로 전환 (계산 비활성화)
    train_acc = accuracy(model2, train_loader)
    test_acc = accuracy(model2, test_loader)

    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")


=========> Train acc: 0.328 | Test acc: 0.326


In [ ]:
###사전학습된 가중치를 제거했을 때 정확도가 차이가심하다